# Testing Grounds for Secure LLM

In [ ]:
# from google import genai

# file_path = 'api_key.txt'     # don't post keys on GitHub
# with open(file_path, 'r', encoding='utf-8') as file:
#     api_key_str = file.read()

# client = genai.Client(api_key=api_key_str)

# response = client.models.generate_content(
#     model="gemini-2.5-flash-lite",
#     contents="How do I launch a secure enclave using Azure",
# )

# print(response.text)

In [16]:
# print(response)

In [17]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

# Generate RSA key pair
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)

# Get public key
public_key = private_key.public_key()

# Serialize private key to PEM format
private_pem = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

# Serialize public key to PEM format
public_pem = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

# Save to files
with open('private_key.pem', 'wb') as f:
    f.write(private_pem)

with open('public_key.pem', 'wb') as f:
    f.write(public_pem)

print("RSA keys generated successfully!")

RSA keys generated successfully!


In [4]:
from cryptography.hazmat.primitives.asymmetric import ed25519
from cryptography.hazmat.primitives import serialization

# Generate keys
private_key = ed25519.Ed25519PrivateKey.generate()
public_key = private_key.public_key()

# Sign a message
message = b"Hello, this is my message!"
signature = private_key.sign(message)

print(f"Signature (hex): {signature.hex()}")
print(f"Signature length: {len(signature)} bytes")

# Verify the signature
try:
    public_key.verify(signature, message)
    print("✓ Signature is valid!")
except Exception as e:
    print(f"✗ Signature verification failed: {e}")

# Try with tampered message
tampered_message = b"Hello, this is a different message!"
try:
    public_key.verify(signature, tampered_message)
    print("✓ Signature is valid!")
except:
    print("✗ Signature verification failed (as expected)")

Signature (hex): db742e0d251b4258d8c366549e3b10fad3cd39f7c76e20fc645a3880c79a6076866e50fa8637b3bb5c301d66d18a63493c44e5cfa601bb881d2e3207ba451709
Signature length: 64 bytes
✓ Signature is valid!
✗ Signature verification failed (as expected)


In [6]:

from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

# Initialize the engines
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

text = "My name is John Smith and I live in Nebraska, help me get an anonymous request with johnsmit@gmail.com."

# Analyze the text to detect PII
analyzer_results = analyzer.analyze(text=text, language='en')

# Anonymize the text
anonymized_text = anonymizer.anonymize(text=text, analyzer_results=analyzer_results).text

print(f"Original Text: {text}")
print(f"Anonymized Text: {anonymized_text}")

Original Text: My name is John Smith and I live in Nebraska, help me get an anonymous request with johnsmit@gmail.com.
Anonymized Text: My name is <PERSON> and I live in <LOCATION>, help me get an anonymous request with <EMAIL_ADDRESS>.
